In [1]:
import pandas as pd
import numpy as np
import allel
import moments.LD

### Preprocessing of 1000G VCF files
1. Apply strict mask
    - Some sites are not reliably mapped in hg38 and needs to be excluded.
    - These regions include: repeat regions, pseudogenes, centromeric, and telomeric regions.
2. Select only biallelic SNP sites
    - A VCF can contain mult-allelic SNPs and structure variants (indels)
    - The biallelic SNP sites are the easiest to work with, and most studies focus on them.

In [ ]:
%%sh
### in bash, preprocess using bcftools
### replace variable path
chr22_vcf_path='/home/alouette/projects/ctb-sgravel/data/1000G/20201028_CCDG_14151_B01_GRM_WGS_2020-08-05_chr22.recalibrated_variants.vcf.gz'
strict_mask_1000G='/home/alouette/projects/ctb-sgravel/data/1000G/masks_and_annotations/20160622.allChr.mask.bed'
module load bcftools
bcftools --version | head -1
### -Oz output compressed VCF format
### -m2 -M2 -v to only view biallellic SNPs
### -R subset VCF from region bed file
bcftools view -Oz -m2 -M2 -v snps -R ${strict_mask_1000G} ${chr22_vcf_path} -o "chr22.strict_masked.vcf.gz"

bcftools 1.11


bcftools manual: http://samtools.github.io/bcftools/bcftools.html#expressions